<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=269564490" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kaggle/input/mooccubex-dataset/concept-paper.txt
/kaggle/input/mooccubex-dataset/translated_com

In [2]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-v4/user_course_v4.csv'
df_user_video_final_path = '/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv'

In [3]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg, regexp_replace
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/21 02:07:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [5]:
df_user_video=spark.read.json(BASE_DIR + "/user-video.json")
df_user_video.show()

+--------------------+-------+
|                 seq|user_id|
+--------------------+-------+
|[{[{190.0, 158843...|  U_112|
|[{[{506.0, 160406...|  U_150|
|[{[{4556.8, 15981...|  U_172|
|[{[{109.0, 159883...|  U_189|
|[{[{64.747, 15821...|  U_197|
|[{[{206.8, 159814...|  U_382|
|[{[{598.978027343...|  U_514|
|[{[{9.388, 160075...|  U_670|
|[{[{99.5, 1580808...|  U_783|
|[{[{454.848426, 1...|  U_796|
|[{[{231.0, 160258...|  U_817|
|[{[{567.2, 160078...|  U_981|
|[{[{595.0, 157961...| U_1071|
|[{[{9.7, 15989332...| U_1073|
|[{[{324.4, 158791...| U_1193|
|[{[{924.9, 158436...| U_1206|
|[{[{36.5, 1580867...| U_1323|
|[{[{391.5, 160091...| U_1361|
|[{[{130.4, 159765...| U_1368|
|[{[{1214.0, 16037...| U_1418|
+--------------------+-------+
only showing top 20 rows



In [6]:
df_user_video.printSchema()

root
 |-- seq: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- segment: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_point: double (nullable = true)
 |    |    |    |    |-- local_start_time: long (nullable = true)
 |    |    |    |    |-- speed: double (nullable = true)
 |    |    |    |    |-- start_point: double (nullable = true)
 |    |    |-- video_id: string (nullable = true)
 |-- user_id: string (nullable = true)



In [7]:
from pyspark.sql.functions import col, explode, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Giải nén `seq` để mỗi dòng có một `video_id`
df_flatten = df_user_video.withColumn("seq", explode(col("seq")))

# Giải nén tiếp `segment` để mỗi dòng là một đoạn xem video
df_flatten = df_flatten.withColumn("segment", explode(col("seq.segment")))

# Chọn các cột cần thiết
df_flatten = df_flatten.select(
    col("user_id"),
    col("seq.video_id").alias("video_id"),
    col("segment.start_point"),
    col("segment.end_point"),
    col("segment.speed"),
    col("segment.local_start_time")
)
df_flatten.show(truncate=True)

+-------+---------+-----------+---------+-----+----------------+
|user_id| video_id|start_point|end_point|speed|local_start_time|
+-------+---------+-----------+---------+-----+----------------+
|  U_112|V_1395633|      130.0|    190.0|  1.0|      1588431144|
|  U_112|V_1395633|      220.0|    250.0|  1.0|      1588431234|
|  U_112|V_1395633|      478.8|    508.8|  1.0|      1588437359|
|  U_112|V_1395633|      655.1|   692.55| 1.25|      1588437514|
|  U_112|V_1395635|      135.0|    170.0|  1.0|      1588438045|
|  U_112|V_1395636|      128.3|    180.8|  1.5|      1588438278|
|  U_112|V_1395639|      100.0|   106.25| 1.25|      1588438980|
|  U_112|V_1395639|      180.0|   186.25| 1.25|      1588439045|
|  U_112|V_6210799|        9.0|    22.25| 1.25|      1601012656|
|  U_112|V_6210799|       28.0|    271.5| 1.25|      1601012671|
|  U_112|V_6210799|      279.0|    331.5|  1.5|      1601012841|
|  U_112|V_6210799|      346.0|    356.0|  1.5|      1601012886|
|  U_112|V_6210800|      

In [8]:
from pyspark.sql.functions import from_unixtime

df_flatten = df_flatten.withColumn("local_start_time", from_unixtime("local_start_time", "yyyy-MM-dd HH:mm:ss"))
df_flatten.show(truncate=False)

+-------+---------+-----------+---------+-----+-------------------+
|user_id|video_id |start_point|end_point|speed|local_start_time   |
+-------+---------+-----------+---------+-----+-------------------+
|U_112  |V_1395633|130.0      |190.0    |1.0  |2020-05-02 14:52:24|
|U_112  |V_1395633|220.0      |250.0    |1.0  |2020-05-02 14:53:54|
|U_112  |V_1395633|478.8      |508.8    |1.0  |2020-05-02 16:35:59|
|U_112  |V_1395633|655.1      |692.55   |1.25 |2020-05-02 16:38:34|
|U_112  |V_1395635|135.0      |170.0    |1.0  |2020-05-02 16:47:25|
|U_112  |V_1395636|128.3      |180.8    |1.5  |2020-05-02 16:51:18|
|U_112  |V_1395639|100.0      |106.25   |1.25 |2020-05-02 17:03:00|
|U_112  |V_1395639|180.0      |186.25   |1.25 |2020-05-02 17:04:05|
|U_112  |V_6210799|9.0        |22.25    |1.25 |2020-09-25 05:44:16|
|U_112  |V_6210799|28.0       |271.5    |1.25 |2020-09-25 05:44:31|
|U_112  |V_6210799|279.0      |331.5    |1.5  |2020-09-25 05:47:21|
|U_112  |V_6210799|346.0      |356.0    |1.5  |2

In [9]:
from pyspark.sql.functions import regexp_replace, col
df_cleaned = df_flatten.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

df_cleaned.show(5, truncate=False)


+-------+--------+-----------+---------+-----+-------------------+
|user_id|video_id|start_point|end_point|speed|local_start_time   |
+-------+--------+-----------+---------+-----+-------------------+
|112    |1395633 |130.0      |190.0    |1.0  |2020-05-02 14:52:24|
|112    |1395633 |220.0      |250.0    |1.0  |2020-05-02 14:53:54|
|112    |1395633 |478.8      |508.8    |1.0  |2020-05-02 16:35:59|
|112    |1395633 |655.1      |692.55   |1.25 |2020-05-02 16:38:34|
|112    |1395635 |135.0      |170.0    |1.0  |2020-05-02 16:47:25|
+-------+--------+-----------+---------+-----+-------------------+
only showing top 5 rows



In [10]:
df_cleaned.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- video_id: long (nullable = true)
 |-- start_point: double (nullable = true)
 |-- end_point: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- local_start_time: string (nullable = true)



In [11]:
df_cleaned = df_cleaned.filter(col("start_point") <= col("end_point"))


**video_duration:** Tổng thời gian dài nhất mà video đã được xem (last_end_time). Đây là thời gian tối đa của video mà người dùng đã từng xem.<br>
**rewind_time:**	Tổng thời gian người dùng tua lại video (khi start_point của đoạn hiện tại nhỏ hơn end_point của đoạn trước đó).<br>
**fast_forward_time:**	Tổng thời gian người dùng tua tới trong video (khi start_point của đoạn hiện tại lớn hơn end_point của đoạn trước đó).<br>
**first_local_start_time:**	Thời điểm đầu tiên mà người dùng bắt đầu xem video (local_start_time của lần xem đầu tiên).<br>
**actual_watch_time:**	Tổng thời gian thực tế mà người dùng đã xem video (tổng các đoạn end_point - start_point). Không tính việc video bị tua nhanh hay chậm.<br>
**weighted_watch_time:**	Tổng thời gian xem có tính tốc độ (lấy mỗi đoạn thời gian thực tế chia cho tốc độ tương ứng, rồi cộng lại). <br>Công thức: \
weighted_watch_time = **sum(**(end_point - start_point) / speed**)**  . 	<br>
Giá trị này phản ánh thời gian video đã chạy nếu xem với tốc độ gốc (1.0x).
**average_speed:**	Tốc độ xem trung bình của người dùng trên video. <br>
Công thức: *actual_watch_time / weighted_watch_time.* <br>
Nếu bằng 1.0, người dùng xem với tốc độ gốc. Nếu lớn hơn 1.0, người dùng xem nhanh hơn. Nếu nhỏ hơn 1.0, người dùng xem chậm lại.

In [12]:
from pyspark.sql.functions import col, min, max, count, sum, lag, first, when

from pyspark.sql.window import Window

# Tạo cửa sổ để so sánh với điểm trước đó của user trên cùng video
window_spec = Window.partitionBy("user_id", "video_id").orderBy("local_start_time")

# Tạo cột prev_end_point để so sánh với điểm trước đó của video
df_lagged = df_cleaned.withColumn("prev_end_point", lag("end_point").over(window_spec))

# Tính toán các giá trị mong muốn
df_user_video_final = df_lagged.groupBy("user_id", "video_id").agg(
    round(max("end_point"), 2).alias("video_duration"),      # Tổng thời gian của video (giá trị lớn nhất của end_point)
    sum(
        when(col("prev_end_point") > col("start_point"),
             round(col("prev_end_point") - col("start_point"), 2))  # Nếu prev_end > start -> tua lại
        .otherwise(0)
    ).alias("rewind_time"),
    sum(
        when(col("start_point") > col("prev_end_point"),
             round(col("start_point") - col("prev_end_point"), 2))  # Nếu start > prev_end -> tua tới
        .otherwise(0)
    ).alias("fast_forward_time"),
    first("local_start_time").alias("first_local_start_time"),  # Lần đầu tiên xem video
    round(sum(col("end_point") - col("start_point")), 2).alias("actual_watch_time"),  # Tổng thời gian thực tế đã xem
    round(sum((col("end_point") - col("start_point")) / col("speed")), 2).alias("weighted_watch_time"),  # Tổng thời gian thực / speed
)

# Tính tốc độ xem trung bình
df_user_video_final = df_user_video_final.withColumn(
    "average_speed", round(col("actual_watch_time") / col("weighted_watch_time"), 2)
)

df_user_video_final.show(truncate=True)


+-------+--------+--------------+-----------+-----------------+----------------------+-----------------+-------------------+-------------+
|user_id|video_id|video_duration|rewind_time|fast_forward_time|first_local_start_time|actual_watch_time|weighted_watch_time|average_speed|
+-------+--------+--------------+-----------+-----------------+----------------------+-----------------+-------------------+-------------+
|    112| 1395635|         170.0|        0.0|              0.0|   2020-05-02 16:47:25|             35.0|               35.0|          1.0|
|    172| 6432959|        4556.8|        0.0|              0.0|   2020-08-23 14:23:14|              4.7|                4.7|          1.0|
|    197| 1358557|        857.94|     344.24|           433.67|   2020-02-21 03:57:27|             40.0|               40.0|          1.0|
|    197| 1358708|         535.1|        0.0|             52.7|   2020-03-26 02:05:02|            477.1|             318.07|          1.5|
|    382| 1258459|         

In [13]:
uv_null_ratio = df_user_video_final.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_video_final.columns
])
uv_null_ratio.show()

+-------+--------+--------------+-----------+-----------------+----------------------+-----------------+-------------------+-------------+
|user_id|video_id|video_duration|rewind_time|fast_forward_time|first_local_start_time|actual_watch_time|weighted_watch_time|average_speed|
+-------+--------+--------------+-----------+-----------------+----------------------+-----------------+-------------------+-------------+
|      0|       0|             0|          0|                0|                     0|                0|                  0|          581|
+-------+--------+--------------+-----------+-----------------+----------------------+-----------------+-------------------+-------------+



In [14]:
df_course = spark.read.json(BASE_DIR + '/course.json')

In [15]:
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)

df_video_course_map.show()

+---------+--------+
|course_id|video_id|
+---------+--------+
| C_584313|   V_849|
| C_584313|   V_850|
| C_584313|   V_851|
| C_584313|   V_857|
| C_584313|   V_859|
| C_584313|   V_861|
| C_584313|   V_862|
| C_584313|   V_867|
| C_584313|   V_868|
| C_584313|   V_869|
| C_584313|   V_870|
| C_584313|   V_871|
| C_584313|   V_877|
| C_584313|   V_878|
| C_584313|   V_879|
| C_584313|   V_880|
| C_584313|   V_881|
| C_584313|   V_883|
| C_584313|   V_884|
| C_584313|   V_885|
+---------+--------+
only showing top 20 rows



In [16]:
df_video_course_map_clean = df_video_course_map.withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

df_video_course_map_clean.show()

+---------+--------+
|course_id|video_id|
+---------+--------+
|   584313|     849|
|   584313|     850|
|   584313|     851|
|   584313|     857|
|   584313|     859|
|   584313|     861|
|   584313|     862|
|   584313|     867|
|   584313|     868|
|   584313|     869|
|   584313|     870|
|   584313|     871|
|   584313|     877|
|   584313|     878|
|   584313|     879|
|   584313|     880|
|   584313|     881|
|   584313|     883|
|   584313|     884|
|   584313|     885|
+---------+--------+
only showing top 20 rows



In [17]:
df_video = spark.read.json(BASE_DIR + "/video.json")
df_video.show()

+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|                ccid|                 end|                  name|               start|                                text|
+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|0001603F826A3DDA9...|[4.255, 8.119, 10...|                 Video|[1.031, 7.095, 8....|[第二个就是短助记符在生成上面。有...|
|0003DB14A14A53279...|[12.581, 17.38, 1...|                 Video|[11.799, 12.581, ...|  [大家好, 今天我们讨论一维系统辐...|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|        家禽的消化系统|[19.2, 20.84, 23....|   [各位同学 大家好, 我是来自中国...|
|00059EBD1371A6959...|[23.14, 194.458, ...|    1.2 计算机硬件系统|[19.372, 190.522,...| [大家好 本视频介绍计算机硬件系统...|
|0005D1DC01B4EF8F9...|[25.08, 26.775, 2...|      什么是供应链网络|[24.33, 25.081, 2...|   [大家好, 今天我们学习的知识点,...|
|0007919ED7652A779...|[7.26, 9.97, 11.5...|    1.2 英语文字的起源|[6.29, 7.31, 10.0...|  [同学们好, 今天我们

In [18]:
from pyspark.sql.functions import array_max
df_video = (
    df_video
    .select(
        "ccid", 
        array_max("end").alias("video_duration")
    )
)


In [19]:
df_video.show()

+--------------------+--------------+
|                ccid|video_duration|
+--------------------+--------------+
|0001603F826A3DDA9...|       509.095|
|0003DB14A14A53279...|       367.336|
|0004A5C6F07E369D9...|       492.315|
|00059EBD1371A6959...|       630.757|
|0005D1DC01B4EF8F9...|        408.75|
|0007919ED7652A779...|        506.57|
|0007C4A5229EE3CF9...|       1190.78|
|0008C8CC056F4EAC9...|       130.309|
|000948D39EA6B0A69...|        421.32|
|0009510AE8068CBA9...|        622.68|
|000A1158B61C13B79...|       1281.36|
|000A40E1D226AEE09...|       255.465|
|000A5EE0C695E8EF9...|         553.8|
|000BD50F6730291F9...|       1328.56|
|000E08952A010F3D9...|         590.0|
|000F83F0945DC9D79...|        659.68|
|0012B03A05345F229...|         17.18|
|0012D2CEBDFED7729...|        623.04|
|00137081D7B0E4219...|        530.71|
|00137797685039299...|       351.188|
+--------------------+--------------+
only showing top 20 rows



In [20]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
# Parse mapping file
df_video_id_ccid_v2 = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
df_video_id_ccid_v2.show(5, truncate=False)

+--------+--------------------------------+
|video_id|ccid                            |
+--------+--------------------------------+
|V_234845|0000363DB5B6E0869C33DC5901307461|
|V_234876|0000363DB5B6E0869C33DC5901307461|
|V_234907|0000363DB5B6E0869C33DC5901307461|
|V_293392|0000363DB5B6E0869C33DC5901307461|
|V_293445|0000363DB5B6E0869C33DC5901307461|
+--------+--------------------------------+
only showing top 5 rows



In [21]:
video_info = df_video_id_ccid_v2.join(df_video, on=['ccid'])
video_info_cleaned = video_info.withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)
video_info_cleaned.show(truncate=False)


+--------------------------------+--------+--------------+
|ccid                            |video_id|video_duration|
+--------------------------------+--------+--------------+
|035503AF2FB5812C9C33DC5901307461|125885  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|126437  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|126621  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|681656  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|793328  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|793761  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|794195  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|1260909 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2357873 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2358059 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2612910 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2748021 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2748207 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2971514 |1051.907     

In [22]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course_clean = df_user_course.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
)

df_user_course_clean.show()

+-------+---------+-----------+----------+----------+-------------+-------------------+----------+
|user_id|course_id|certificate|start_date|  end_date|duration_days|        enroll_time|remain_day|
+-------+---------+-----------+----------+----------+-------------+-------------------+----------+
|  10000|  2033958|        1.0|2020-09-04|2020-12-31|        118.0|2020-10-27 09:07:30|      65.0|
|1000129|  1925903|        1.0|2020-08-12|2020-12-31|        141.0|2020-09-06 14:37:17|     116.0|
|1000129|  1992970|        1.0|2020-08-27|2020-12-31|        126.0|2020-09-06 14:45:37|     116.0|
|1000129|   680884|        1.0|2020-09-01|2020-12-31|        121.0|2020-10-16 10:41:09|      76.0|
|1000342|   697791|        0.0|2020-01-20|2020-07-31|        193.0|2020-02-28 10:27:22|     154.0|
|1000663|  2328510|        0.0|2020-11-18|2021-02-28|        102.0|2020-11-27 22:41:36|      93.0|
| 100071|   696601|        1.0|2019-12-25|2020-06-30|        188.0|2020-04-15 08:51:39|      76.0|
|1000826| 

In [23]:
count = df_user_course_clean.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_course_clean.columns
])
count.show(truncate=False)

+-------+---------+-----------+----------+--------+-------------+-----------+----------+
|user_id|course_id|certificate|start_date|end_date|duration_days|enroll_time|remain_day|
+-------+---------+-----------+----------+--------+-------------+-----------+----------+
|0      |0        |4939141    |4939141   |4939141 |4939141      |0          |4939141   |
+-------+---------+-----------+----------+--------+-------------+-----------+----------+



# Thử Nghiệm 

In [24]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType, TimestampType

# 1. Chuẩn bị Dữ liệu Video (Không đổi)
df_video_durations = video_info_cleaned.select(
    F.col("video_id"),
    F.col("video_duration")
).distinct()

# 2. Chuẩn bị Dữ liệu Khóa học (Không đổi)
df_course_video_details = df_video_course_map_clean.join(
    df_video_durations,
    "video_id",
    "left"
)
df_course_stats = df_course_video_details.groupBy("course_id").agg(
    F.collect_list("video_id").alias("course_video_list"),
    F.sum("video_duration").alias("total_video_duration"),
    F.count("video_id").alias("num_videos_in_course")
)

# 3. Chuẩn bị Dữ liệu Xem của Người dùng-Khóa học
df_user_watch_by_course = df_user_video_final.join(
    df_video_course_map_clean,
    "video_id",
    "inner"
)

# --- BẮT ĐẦU TINH CHỈNH 1 ---
# Đổi tên 'video_duration' (từ log) để tránh xung đột
# Cột này (từ user-video.json) đáng tin cậy hơn để tính tỷ lệ hoàn thành sự kiện
df_user_watch_events = df_user_watch_by_course.withColumnRenamed("video_duration", "log_video_duration")

# Join với duration "chính thức" (từ video.json)
df_user_watch_events = df_user_watch_events.join(
    df_video_durations,
    "video_id",
    "left"
)

# Tính 'video_completion_rate_event' DÙNG 'log_video_duration' (đáng tin cậy hơn)
df_user_watch_events = df_user_watch_events.withColumn(
    "video_completion_rate_event",
    F.when(F.col("log_video_duration") > 0, F.col("actual_watch_time") / F.col("log_video_duration"))
     .otherwise(0)
).withColumn(
    "is_rewatch",
    F.when(F.col("rewind_time") > 0, 1).otherwise(0)
)
# --- KẾT THÚC TINH CHỈNH 1 ---


# Tổng hợp hành vi của người dùng (Không đổi)
df_user_course_watch_stats = df_user_watch_events.groupBy("user_id", "course_id").agg(
    F.collect_list("video_id").alias("user_video_list_with_duplicates"),
    F.sum("actual_watch_time").alias("total_watched_duration"),
    F.avg("actual_watch_time").alias("avg_watched_duration"),
    F.avg("average_speed").alias("avg_speed"),
    F.count("video_id").alias("num_sessions"),
    F.countDistinct("video_id").alias("num_videos_watched"),
    F.sum("is_rewatch").alias("num_rewatches"),
    F.avg("video_completion_rate_event").alias("avg_completion_rate_per_video"), # (Giờ sẽ tính đúng)
    F.min("first_local_start_time").alias("first_local_start_time")
).withColumn(
    "user_video_list", F.array_distinct("user_video_list_with_duplicates")
).drop("user_video_list_with_duplicates")


# 4. Gộp và Tính toán (Bước cuối cùng)
df_final_agg = df_user_course_clean.join(
    df_course_stats,
    "course_id",
    "left"
).join(
    df_user_course_watch_stats,
    ["user_id", "course_id"],
    "left"
)

# Xử lý giá trị NULL (Không đổi)
numeric_fill_cols = {
    "total_video_duration": 0.0,
    "num_videos_in_course": 0,
    "total_watched_duration": 0.0,
    "avg_watched_duration": 0.0,
    "avg_speed": 0.0,
    "num_sessions": 0,
    "num_videos_watched": 0,
    "num_rewatches": 0,
    "avg_completion_rate_per_video": 0.0
}
df_final_agg = df_final_agg.fillna(numeric_fill_cols)
df_final_agg = df_final_agg.withColumn(
    "course_video_list",
    F.when(F.col("course_video_list").isNull(), F.array().cast(ArrayType(StringType())))
     .otherwise(F.col("course_video_list"))
).withColumn(
    "user_video_list",
    F.when(F.col("user_video_list").isNull(), F.array().cast(ArrayType(StringType())))
     .otherwise(F.col("user_video_list"))
)

# Tính toán các chỉ số cuối cùng
df_final_output_raw = df_final_agg.withColumn(
    "num_common_videos",
    F.col("num_videos_watched")
).withColumn(
    "estimated_watched_in_course",
    F.col("total_watched_duration")
).withColumn(
    "completion_rate_raw", # Đặt tên thô
    F.when(F.col("total_video_duration") > 0, F.col("total_watched_duration") / F.col("total_video_duration"))
     .otherwise(0.0)
).withColumn(
    "unique_videos_ratio_raw", # Đặt tên thô
    F.when(F.col("num_videos_in_course") > 0, F.col("num_videos_watched") / F.col("num_videos_in_course"))
     .otherwise(0.0)
).withColumn(
    "rewatch_ratio_original",
    F.when(F.col("num_sessions") > 0, F.col("num_rewatches") / F.col("num_sessions"))
     .otherwise(0.0)
).withColumn(
    "rewatch_sessions_ratio",
    F.when(
        (F.col("num_sessions") > 0) & (F.col("num_sessions") > F.col("num_videos_watched")),
        (F.col("num_sessions") - F.col("num_videos_watched")) / F.col("num_sessions")
    ).otherwise(0.0)
).withColumn(
    "video_activity_ratio",
    F.when(F.col("num_videos_in_course") > 0, F.col("num_sessions") / F.col("num_videos_in_course"))
     .otherwise(0.0)
).withColumn(
    "days_from_enroll_to_first_watch",
    F.when(
        F.col("first_local_start_time").isNotNull() & F.col("enroll_time").isNotNull(),
        F.datediff(
            F.col("first_local_start_time").cast("date"),
            F.col("enroll_time").cast("date")
        )
    ).otherwise(None)
)

# --- BẮT ĐẦU TINH CHỈNH 2: CHUẨN HÓA (CAPPING) ---
# Giới hạn các giá trị ngoại lệ (outliers) do lỗi dữ liệu
MAX_COMPLETION = 2.0 # Giới hạn tỷ lệ hoàn thành ở 200% (cho phép xem lại 1 lần)
MAX_RATIO = 1.0       # Giới hạn tỷ lệ video ở 100%

df_final_output = df_final_output_raw.withColumn(
    "completion_rate",
    F.when(F.col("completion_rate_raw") > MAX_COMPLETION, MAX_COMPLETION)
     .otherwise(F.col("completion_rate_raw"))
).withColumn(
    "unique_videos_ratio",
    F.when(F.col("unique_videos_ratio_raw") > MAX_RATIO, MAX_RATIO)
     .otherwise(F.col("unique_videos_ratio_raw"))
)
# --- KẾT THÚC TINH CHỈNH 2 ---


# Sắp xếp lại các cột (Giữ nguyên)
final_columns = [
    "user_id", "course_id", "certificate", "enroll_time",
    "start_date", "end_date", "duration_days", "remain_day",
    "first_local_start_time",
    "days_from_enroll_to_first_watch",
    "user_video_list", "avg_watched_duration", "avg_speed",
    "num_videos_watched", "num_sessions", "total_watched_duration",
    "course_video_list", "total_video_duration",
    "num_common_videos", "estimated_watched_in_course", "completion_rate", # Đã chuẩn hóa
    "unique_videos_ratio", # Đã chuẩn hóa
    "rewatch_ratio_original",
    "rewatch_sessions_ratio",
    "avg_completion_rate_per_video", # Đã sửa logic
    "video_activity_ratio"
]
df_final_output = df_final_output.select(final_columns)


# THÊM MỚI: Vòng lặp định dạng % (Thêm vào cuối)
rate_cols = [
    "completion_rate",
    "unique_videos_ratio",
    "rewatch_ratio_original",
    "rewatch_sessions_ratio",
    "avg_completion_rate_per_video",
    "video_activity_ratio"
]

for c in rate_cols:
    df_final_output = df_final_output.withColumn(
        c,
        F.concat(F.round(F.col(c) * 100, 2).cast("string"), F.lit("%"))
    )

# Hiển thị kết quả (đã tinh chỉnh và chuẩn hóa)
df_final_output.orderBy(F.col("completion_rate").desc_nulls_last()).show(20, truncate=True)

25/10/21 02:12:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+---------+-----------+-------------------+----------+----------+-------------+----------+----------------------+-------------------------------+--------------------+--------------------+------------------+------------------+------------+----------------------+--------------------+--------------------+-----------------+---------------------------+---------------+-------------------+----------------------+----------------------+-----------------------------+--------------------+
| user_id|course_id|certificate|        enroll_time|start_date|  end_date|duration_days|remain_day|first_local_start_time|days_from_enroll_to_first_watch|     user_video_list|avg_watched_duration|         avg_speed|num_videos_watched|num_sessions|total_watched_duration|   course_video_list|total_video_duration|num_common_videos|estimated_watched_in_course|completion_rate|unique_videos_ratio|rewatch_ratio_original|rewatch_sessions_ratio|avg_completion_rate_per_video|video_activity_ratio|
+--------+------

In [25]:
# Lọc để chỉ hiển thị những hàng có thời gian xem lớn hơn 0
df_active_users = df_final_output.filter(F.col("total_watched_duration") > 0)

df_active_users.show()

+-------+---------+-----------+-------------------+----------+----------+-------------+----------+----------------------+-------------------------------+--------------------+--------------------+------------------+------------------+------------+----------------------+--------------------+--------------------+-----------------+---------------------------+---------------+-------------------+----------------------+----------------------+-----------------------------+--------------------+
|user_id|course_id|certificate|        enroll_time|start_date|  end_date|duration_days|remain_day|first_local_start_time|days_from_enroll_to_first_watch|     user_video_list|avg_watched_duration|         avg_speed|num_videos_watched|num_sessions|total_watched_duration|   course_video_list|total_video_duration|num_common_videos|estimated_watched_in_course|completion_rate|unique_videos_ratio|rewatch_ratio_original|rewatch_sessions_ratio|avg_completion_rate_per_video|video_activity_ratio|
+-------+---------

In [26]:
total = df_final_output.count()

null_ratio = df_final_output.select([
    (F.count(F.when(F.col(c).isNull(), 1)) / total * 100).alias(c) for c in df_final_output.columns
])
null_ratio.show()

+-------+---------+-----------------+-----------+-----------------+-----------------+-----------------+-----------------+----------------------+-------------------------------+---------------+--------------------+---------+------------------+------------+----------------------+-----------------+--------------------+-----------------+---------------------------+---------------+-------------------+----------------------+----------------------+-----------------------------+--------------------+
|user_id|course_id|      certificate|enroll_time|       start_date|         end_date|    duration_days|       remain_day|first_local_start_time|days_from_enroll_to_first_watch|user_video_list|avg_watched_duration|avg_speed|num_videos_watched|num_sessions|total_watched_duration|course_video_list|total_video_duration|num_common_videos|estimated_watched_in_course|completion_rate|unique_videos_ratio|rewatch_ratio_original|rewatch_sessions_ratio|avg_completion_rate_per_video|video_activity_ratio|
+-----

In [27]:
# from pyspark.sql import functions as F

# # Tạo một DataFrame mới để chuẩn bị lưu CSV
# # Chuyển đổi các cột mảng thành chuỗi, phân tách bằng dấu '|'
# df_for_csv = df_final_output.withColumn(
#     "user_video_list", 
#     F.concat_ws("|", F.col("user_video_list"))
# ).withColumn(
#     "course_video_list", 
#     F.concat_ws("|", F.col("course_video_list"))
# )

# # Bây giờ tất cả các cột đều là kiểu nguyên thủy, có thể lưu ra CSV
# df_for_csv.coalesce(1).write.mode("overwrite").option("header", "true").csv("/kaggle/working/df_final_output")

# print("Đã lưu file CSV thành công!")

In [28]:
# import shutil

# input_dir = "/kaggle/working/df_final_output"
# output_zip = "/kaggle/working/df_final_output"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_final_output.zip")
